# 4 The data

In [1]:
import os
import numpy as np
import pickle
import random
import re

Read both positive and negative reviews and put it into a large python list.

In [23]:
neg_dir = '../data/neg/'
pos_dir = '../data/pos/'
data = []
step = 0
for filename in os.listdir(neg_dir):
#     step+=1
#     if step>2:
#         break
    path = neg_dir+filename
    with open(path) as f:
        text = f.read().replace('\n','')
        example = {}
        example['label'] = 0
        # Strip out the parse information and the phrase labels---we don't need those here
        text = re.sub(r'\s*(\(\d)|(\))\s*', '', text)
        example['text'] = text[1:]
        data.append(example)
        
for filename in os.listdir(pos_dir):
#     step+=1
#     if step>2:
#         break
    path = pos_dir+filename
    with open(path) as f:
        text = f.read().replace('\n','')
        example = {}
        example['label'] = 1
        # Strip out the parse information and the phrase labels---we don't need those here
        text = re.sub(r'\s*(\(\d)|(\))\s*', '', text)
        example['text'] = text[1:]
        data.append(example)

#Shuffle data
random.seed(1)
random.shuffle(data)

In [27]:
#Training-validation split
training_set = data[:1500]
validation_set = data[1500:]

/newpage
# Sparse Representations

My feature extract function transfer the text into both sparse representation dictionary(named'feature') as described in question and bag-of-words representation(named 'vector') numpy vector(with tons of zeros).

In [47]:
#Write a function to extract bag-of-words features
#The feature_function is modified from Samuel Bowman's Natural Language Processing class in fall 2016.
import collections
import numpy as np

def feature_function(datasets):
    '''Annotates datasets with feature vectors.'''
    
    # Extract vocabulary
    def tokenize(string):
        return string.split()
    
    word_counter = collections.Counter()
    for example in datasets[0]:
        word_counter.update(tokenize(example['text']))
    
    vocabulary = set([word for word in word_counter])
                                
    feature_names = set()
    for i, dataset in enumerate(datasets):
        for example in dataset:
            example['features'] = collections.defaultdict(float)
            
            # Extract features (by name) for one example
            word_counter = collections.Counter(tokenize(example['text']))
            for x in word_counter.items():
                if x[0] in vocabulary:
                    example["features"][x[0]] = x[1]
            
            feature_names.update(example['features'].keys())
                            
    # By now, we know what all the features will be, so we can
    # assign indices to them.
    feature_indices = dict(zip(feature_names, range(len(feature_names))))
    indices_to_features = {v: k for k, v in feature_indices.items()}
    dim = len(feature_indices)
                
    # Now we create actual vectors from those indices.
    for dataset in datasets:
        for example in dataset:
            example['vector'] = np.zeros((dim))
            for feature in example['features']:
                example['vector'][feature_indices[feature]] = example['features'][feature]
    return indices_to_features, feature_indices,dim
    
indices_to_features,feature_indices,dim = feature_function([data])

TypeError: zip argument #2 must support iteration

\newpage
# 6 Support Vector Machine Via Pegasos
## 6.1
Since $\frac{\lambda}{2}\lvert\lvert w\rvert\rvert^2$ is convex and the hinge loss is convex, using the fact provided in the question the subgradient of the objective function is:$
\begin{cases}
\lambda w -y_i x_i & 1-y_i w x_i >0\\
\lambda w & else
\end{cases}
$
Using the stochastic subgradient descent algorithm, the $w$ is updated by:
$
\begin{cases}
w=w-\eta_t(\lambda w -y_i x_i)=(1-\eta_t\lambda)w+\eta_ty_ix_i & 1-y_i w x_i >0\\
w=w-\eta\lambda w= (1-\eta_t\lambda)w& else
\end{cases}
$. This is the same as Pegasos algorithm.

\newpage
## 6.2


In [36]:
def pegasos_vector(data,lambda_reg=0.1):
    t=0
    w=np.zeros(len(data[0]['vector'])) 
    for example in data:
        t+=1
        eta=1.0/(lambda_reg*t)
        y_i = example['label']
        x_i = example['vector']
        if y_i*np.dot(w,x_i)<1:
            w=(1-eta*lambda_reg)*w+eta*y_i*x_i
        else:
            w=(1-eta*lambda_reg)*w
    return w

In [38]:
test_w = pegasos(training_set)

\newpage
## 6.3
Notice: With my feature extraction method, the sparse representation vector is stored as numpy array rather than dictionary. Hence my pegasos DOES NOT suffer the slow down as described in the question.

First verify that this is equivalent.
$$
\begin{split}
W_{t+1}&=(1-\eta_t \lambda)W_t+\frac{\eta_t y_i x_i}{1-\eta_t \lambda}
\end{split}
$$